# datAcron Ontology Traversals

This Jupyter NB serves as validation for the example queries and additionally as navigation and exploration notebook for the datAcron triple store. Where data is saved to CSVs, these CSV files are available in the data/ folder. The tutorial queries are sourced from: 
  1. "SPARQL_queries_example.pdf"
  2. http://ai-group.ds.unipi.gr/datacron_ontology/
  3. Mail by Giorgos Santipantakis with comments to 1.

This notebook replicates and extends the given example queries. The main concepts and properties of the datAcron ontology are depicted in the figure below.

![The datAcron ontology](images/schema_poster.svg)

### Definition of a query function

The query function is defined in the custom TripleStoreConnectorClass in datacron_connector.py. It allows to easily query both endpoints:
 - http://83.212.239.107:8890/sparql : short 107
 - http://83.212.239.109:3434/sparql : short 109


In [1]:
import pandas as pd
from pandas.io.json import json_normalize, read_json
from SPARQLWrapper import SPARQLWrapper, JSON, XML, RDF
from datacron_connector import TripleStoreConnector

#create two connector objects: one for 107 and one for 109 endpoints.
ts107 = TripleStoreConnector(0)
ts109 = TripleStoreConnector(1)

### Pull all concepts (ref: Page 1 + Giorgos comments)

The first query returns all concepts, for which at least one assertion exists in the triple. By recommendation of Giorgios, this query is preferred:

In [5]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT DISTINCT ?Concept
WHERE{
    ?Concept a owl:Class
     }
"""

df = ts107.query(qry)
df = ts107.clean(df)
df.head(5)


,Concept
0,Thing
1,Nothing
2,AntiPollution_Vessel
3,DepartureLeg
4,ATSRoute


### Pull all properties

This query gets all properties, for which at least one instance exists in this graph.
(query improved by Giorgios).

I query both endpoints and compare results to get the properties that are only available in the 107 store.


In [21]:
qry = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?property 
WHERE {
    {?property a owl:ObjectProperty} UNION { ?Property a owl:DataProperty}
} ORDER BY ?property
"""

#Lets pull the properties from both endpoints and compare differences.
df1 = ts107.clean(ts107.query(qry))
df2 = ts109.clean(ts109.query(qry))
df3 = df1.merge(df2, how='outer', indicator=True).query('_merge == "left_only"')
df3


,property,_merge
168,http://www.w3.org/ns/ssn/attachedSystem,left_only
169,http://www.w3.org/ns/ssn/deployedOnPlatform,left_only
170,http://www.w3.org/ns/ssn/deployedSystem,left_only
171,http://www.w3.org/ns/ssn/deploymentProcessPart,left_only
172,http://www.w3.org/ns/ssn/detects,left_only
173,http://www.w3.org/ns/ssn/endTime,left_only
174,http://www.w3.org/ns/ssn/featureOfInterest,left_only
175,http://www.w3.org/ns/ssn/forProperty,left_only
176,http://www.w3.org/ns/ssn/hasDeployment,left_only
177,http://www.w3.org/ns/ssn/hasInput,left_only


### Get more information on something (own work)

I am using property paths here to drill through the ontology graph. This seems to work fine with different Classes of the ontology.

In [22]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?s
WHERE {
  ?s rdfs:subClassOf* :Trajectory .
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = ts107.query(qry)
df = ts107.clean(df)
df.head(10)



,s
0,Trajectory
1,SyntheticTrajectory
2,ActualTrajectory
3,IntentedTrajectory
4,RegulatedTrajectory
5,FM_FlightPlanTrajectory
6,OpenTrajectory
7,ClosedTrajectory


For a class or instance of interest, we can use the DESCRIBE syntax to get some more information.

In [73]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

DESCRIBE ?s
WHERE {
  ?s rdfs:subClassOf* :Trajectory .
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = ts107.query(qry)
df = ts107.clean(df)
df.head(10)



,s,p,o
0,RegulatedTrajectory,comment,A trajectory that has been regulated
1,FM_FlightPlanTrajectory,comment,"A sub-class of intented trajectory, which is r..."
2,hasPart,domain,Trajectory
3,ClosedTrajectory,subClassOf,ActualTrajectory
4,ClosedTrajectory,comment,A Trajectory where the destination is reached
5,Trajectory,comment,A trajectory consists of a sequence of tempor...
6,OpenTrajectory,type,http://www.w3.org/2002/07/owl#Class
7,departureOfTrajectory,domain,Trajectory
8,Trajectory,http://purl.org/dc/elements/1.1/creator,datAcron-project
9,Trajectory,type,http://www.w3.org/2002/07/owl#Class


Giorgos recommends also this syntax below and also rdfs:seeAlso

In [35]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

DESCRIBE ?s
WHERE {
  ?s rdfs:isDefinedBy* :Trajectory .
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = ts107.query(qry)
df = ts107.clean(df)
df.head(10)



,s,p,o
0,hasPart,domain,Trajectory
1,Trajectory,comment,A trajectory consists of a sequence of tempor...
2,Trajectory,type,Class
3,departureOfTrajectory,domain,Trajectory
4,Trajectory,http://purl.org/dc/elements/1.1/creator,datAcron-project
5,destinationOfTrajectory,domain,Trajectory
6,IntentedTrajectory,subClassOf,Trajectory
7,RegulatedTrajectory,subClassOf,Trajectory
8,ActualTrajectory,subClassOf,Trajectory
9,SyntheticTrajectory,subClassOf,Trajectory


### Pull sector configs, affected airspaces, time periods and capacities (Page 2)

This query pulls some sector configurations which were selected in the past. For these configurations, it lists 10 affected airspaces, their period of effectivity and the defined capacity.

In [195]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?config ?airspace ?capacity ?start ?end
WHERE {
  ?config a :FM_Configuration ;
          :hasCapacity ?capacity ;
          :configurationOfAirspace ?airspace ;
          dul:hasConstituent/:TimeStart ?start ;
          dul:hasConstituent/:TimeEnd ?end.
}
ORDER BY ?start
LIMIT 10
"""

df = ts107.query(qry)
df = ts107.clean(df)
df.to_csv('data/configs_and_affected_airspaces1.csv')
df.head(5)



,config,airspace,capacity,start,end
0,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-01T23:59:00
1,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-02T23:59:00
2,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-03T23:59:00
3,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-04T23:59:00
4,AirspaceConfiguration_LFMMXCTA_CF1_411,Airspace_LFMMXCTA_411,999,2016-03-31T00:00:00,2016-04-05T23:59:00


### Configurations and their sectors (own work)

If we want to know which configuration ever affected which sector, we can alter the above query: we omit the timestamp and select only DISTINCT values.

In [197]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT * 
WHERE {
  ?config a :FM_Configuration ;
          :hasCapacity ?capacity ;
          :configurationOfAirspace ?airspace .
}
LIMIT 25000
"""

df = ts107.query(qry)
df = ts107.clean(df)
%time df = df.sort_values('airspace')
df.to_csv('data/configs_and_affected_airspaces2.csv')
df.head(5)



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.94 ms


,config,capacity,airspace
0,AirspaceConfiguration_BIRDCTA_CONF1_411,999,Airspace_BIRDCTA_411
1,AirspaceConfiguration_BIRDICTA_CNF1_411,999,Airspace_BIRDICTA_411
2,AirspaceConfiguration_BIRDTOCA_CONF1_411,999,Airspace_BIRDTOCA_411
3,AirspaceConfiguration_DAAACTA_CONF1_411,999,Airspace_DAAACTA_411
4,AirspaceConfiguration_DAAATCTA_CNF1_411,999,Airspace_DAAATCTA_411


RESULT: Technical result: to avoid timeouts, it is better to pull the data first and let Pandas do the sorting. If I include an ORDER BY clause, the query will timeout. Instead, I use ```pandas df.sort_values()``` method. According to Giorgos comment, the same query should work with "ORDER BY" for the 109 endpoint, which I could confirm below:

In [23]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT * 
WHERE {
  ?config a :FM_Configuration ;
          :hasCapacity ?capacity ;
          :configurationOfAirspace ?airspace .
} ORDER BY ?capacity
"""

df = ts107.query(qry)
df = ts107.clean(df)
df.to_csv('data/configs_and_affected_airspaces3_109.csv')
df.head(5)



,config,capacity,airspace
0,AirspaceConfiguration_HECCCTA_CONF1_411,0,Airspace_HECCCTA_411
1,AirspaceConfiguration_EDWWCTAE_E8B_411,10,Airspace_EDWWCTAE_411
2,AirspaceConfiguration_EDWWCTAE_E9A_411,10,Airspace_EDWWCTAE_411
3,AirspaceConfiguration_EDWWCTAS_S2A_411,12,Airspace_EDWWCTAS_411
4,AirspaceConfiguration_LAAACTA_CONF2_411,13,Airspace_LAAACTA_411


### Pull airspaces with capacity != 999 (Page 2)

The following query pulls only those configurations, where a capacity number != 999 was set, by using a FILTER operator.

In [200]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT * 
WHERE {
   ?c a :FM_Configuration ;
      :hasCapacity ?capacity ;
      :configurationOfAirspace ?airspace ;
      dul:hasConstituent/:TimeStart ?start ;
      dul:hasConstituent/:TimeEnd ?end.
   FILTER(?capacity !='999')
}
"""

df = ts107.query(qry)
df = ts107.clean(df)
df = df.sort_values('airspace')
df.to_csv('data/capacity_limited_configs.csv')
df.head(5)



,c,capacity,airspace,start,end
993,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-19T08:00:00,2016-04-19T06:59:00
973,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-13T08:00:00,2016-04-19T08:59:00
974,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-13T08:00:00,2016-04-19T08:59:00
975,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-13T08:00:00,2016-04-19T08:59:00
977,AirspaceConfiguration_EBBUCTA_CE2W2L_411,38,Airspace_EBBUCTA_411,2016-04-19T04:40:00,2016-04-13T08:59:00


### Configs and airspaces with limited capacity: only DISCTINCT values (own work)

I want to know if there is a direct relation between a configuration ID and the resulting airspace capacity. If this is the case, then the following query should only return one row per configuration ID.


In [202]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT * 
WHERE {
   ?config a :FM_Configuration ;
           :hasCapacity ?capacity ;
           :configurationOfAirspace ?airspace .
   FILTER(?capacity !='999')
}
"""

df = ts107.query(qry)
df = ts107.clean(df)
df.to_csv('data/capacity_limited_configs2.csv')
df.iloc[9:14]



,config,capacity,airspace
9,AirspaceConfiguration_EDWWCTAE_E6C_411,30,Airspace_EDWWCTAE_411
10,AirspaceConfiguration_EDWWCTAE_E7A_411,15,Airspace_EDWWCTAE_411
11,AirspaceConfiguration_EDWWCTAE_E7A_411,30,Airspace_EDWWCTAE_411
12,AirspaceConfiguration_EDWWCTAE_E8A_411,15,Airspace_EDWWCTAE_411
13,AirspaceConfiguration_EDWWCTAE_E8A_411,30,Airspace_EDWWCTAE_411


RESULT SEMANTICAL: as we can see with airspace configuration AirspaceConfiguration_EDWWCTAE_E7A_411 (EDWW is Bremen Radar), there are two different capacities for a single configuration. Therefore, the assumption that a direct dependency from configuration --> capacity exists, does not hold. 

It has to be noted though, that, looking at spanish sector configs, I could not find duplicate entries. TODO let Jose Manuel confirm that.

Giorgios recommended the following query, where we can see that there a couple of configurations where the capacity was not unique:



In [36]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

PREFIX : <http://www.datacron-project.eu/datAcron#>
SELECT ?config (COUNT(?capacity) as ?count)
WHERE {
  ?config a :FM_Configuration ;
          :hasCapacity ?capacity .
}
GROUP BY ?config
ORDER BY DESC(?count)

"""

df = ts107.query(qry)
df = ts107.clean(df)
df.to_csv('data/capacity_limited_configs3.csv')
df.head(20)



,config,count
0,AirspaceConfiguration_LFSBTMA_ALL_411,6
1,AirspaceConfiguration_LSAZUTA_1UA_411,6
2,AirspaceConfiguration_LSAZUTA_2UA_411,6
3,AirspaceConfiguration_LSAZUTA_3UC_411,6
4,AirspaceConfiguration_LSAZUTA_5UA_411,6
5,AirspaceConfiguration_LSAZUTA_4UA_411,6
6,AirspaceConfiguration_LSAZUTA_4UB_411,6
7,AirspaceConfiguration_LSAZUTA_2UB_411,6
8,AirspaceConfiguration_LSAZUTA_3UA_411,6
9,AirspaceConfiguration_EDWWCTAE_E8A_411,4


### Restricting the sector configuration query to a specific airspace (Page 3 + Giorgos comments)

PROBLEM (see Giorgos' comment below)

The following query pulls all configurations that belong to the Airspace Airspace_LSAGCTA (Geneva Control Area). Unfortunately, it returns more than 10000 results, most of them are illogical, because the airspace is configured several times at the same time, which semantically makes no sense according to my understanding of the ATC environment.


In [74]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT * 
WHERE {
  ?config a :FM_Configuration ;
          :hasCapacity ?capacity ;
          :configurationOfAirspace  :Airspace_LSAGCTA_411 ;
          dul:hasConstituent/:TimeStart ?start ;
          dul:hasConstituent/:TimeEnd ?end.
}
"""

df = ts107.query(qry)
df = ts107.clean(df)
df = df.sort_values('start')
df.to_csv('data/configs_of_geneva.csv')
len(df)



10000

In [36]:
df.head(10)

,config,capacity,start,end
0,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-01T03:54:00
565,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T10:44:00
566,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T10:49:00
567,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T10:55:00
568,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T10:59:00
569,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T11:04:00
570,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T14:51:00
571,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T14:54:00
572,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T15:09:00
573,AirspaceConfiguration_LSAGCTA_I1A_411,40,2016-04-01T00:00:00,2016-04-18T15:14:00


PROBLEM:

Instead of 2049 results as according to the tutorial, I get over 10000 configurations for the Geneva airspace?  And they all overlap in time?


NOTE
The syntactic subtlety: obviously, 'Airspace_LSAGCTA_411' is not the name but the ID of the Airspace in the triple store. Therefore, we do not need to search for a name and then combine the queries; it is sufficent to enter the Id directly with the ":" syntax. 


Giorgios' comments:

The following query checks how often the sectors were opened and closed:


In [43]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
SELECT ?config (COUNT(?start) as ?count)
WHERE {
  ?config a :FM_Configuration ;
          :hasCapacity ?capacity ;
          :configurationOfAirspace :Airspace_LSAGCTA_411 ;
          dul:hasConstituent/:TimeStart ?start ;
          dul:hasConstituent/:TimeEnd ?end.
}
GROUP BY ?config
"""

df = ts107.query(qry)
df = ts107.clean(df)
df.to_csv('data/configs_of_geneva2.csv')
df.head(10)



,config,count
0,AirspaceConfiguration_LSAGCTA_I1A_411,2937796
1,AirspaceConfiguration_LSAGCTA_I3A_411,26244
2,AirspaceConfiguration_LSAGCTA_I2A_411,54756
3,AirspaceConfiguration_LSAGCTA_I2B_411,3175524


... "this made me check the raw data and I noticed that some cases "open" and "close" multiple times in the same day, e.g. you can find:

`
...
./OpeningScheme.cos:19/04/2016;LSAGCTA;05:05;05:19;I2B;T
./OpeningScheme.cos:19/04/2016;LSAGCTA;05:20;05:24;I2B;T
./OpeningScheme.cos:19/04/2016;LSAGCTA;05:25;05:36;I2B;T
...
`

The reason and semantics of these cases is not obvious to me either, and it is quite peculiar. We could also ask about it Jose Manuel."


This is a TODO Item --> Jörg


### Further inspect a specific airspace: drilling down to the airblocks (own work)


Comment by Giorgos: please consider using the 109 endpoint for this, as the 107 is not able to deserialize geometries to WKT.

By traversing the ontology, we should be able to drill down to the airblocks and get their properties.
The query below shows us that our airspace of interest, Airspace_LSAGCTA_411 consist of three sub-airspaces: Airspace_LSAGE_411, Airspace_LSAGN_411, Airspace_LSAGS_411.

RESULT SEMANTICAL:

LSAGN is Geneva North, LSAGE is Geneva East and LSAGS is Geneva South. The appendix _411 refers to the AIRAC cycle of this airspace. Reminder: the shape of an airspace may change at the change of an airac cycle. Therefore, the design decision to add the appendix _411 directly to the name of the airspace guarantees that there should be only one geometry per airspace ID.

In [24]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?p ?o
WHERE {
  :Airspace_LSAGCTA_411 ?p ?o .
}
LIMIT 2000
"""

df = ts109.query(qry)
df = ts109.clean(df)
df.to_csv('data/airspace_properties.csv')
df.head(10)

,p,o
0,type,FM_Airspace
1,hasPart,Airspace_LSAGE_411
2,hasPart,Airspace_LSAGN_411
3,hasPart,Airspace_LSAGS_411


Lets inspect these sub-airspaces further. The following query on 107 will return wrong airblock strings in form of `Airblock_LSAGE_LSAGE_301LS_75_245_75_245`, where the height level 75_245 is attached twice to the name. According Giorgos, I should use the 109 store until the 107 store is cleared of these obsolete triples. Below, I query both endpoints for documentation purposes.

In [61]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT  ?s ?p ?o
WHERE { 
  VALUES ?s { :Airspace_LSAGE_411 :Airspace_LSAGN_411 :Airspace_LSAGS_411}
  VALUES ?p { dul:hasPart}
  ?s dul:hasPart ?o. 
}

LIMIT 2000
"""

# to check spanish airspace blocks, give values :Airspace_LECMASU_411 
#              :Airspace_LECMBLL_411 :Airspace_LECMBLU_411 
#              :Airspace_LECMCJL_411 :Airspace_LECMCJU_411 :Airspace_LECMDGL_411
#              :Airspace_LECMDGU_411 :Airspace_LECMPAL_411

blocks107 = ts107.query(qry)
blocks107 = ts107.clean(blocks107)
blocks107.to_csv('data/airspace_inspection.csv')

print(blocks107.head(10))
print(" ")

blocks109 = ts109.query(qry)
blocks109 = ts109.clean(blocks109)
blocks109.to_csv('data/airspace_inspection.csv')

print(blocks109.head(10))

                    s        p                                           o
0  Airspace_LSAGE_411  hasPart    Airblock_LSAGE_LSAGE_301LS_75_245_75_245
1  Airspace_LSAGE_411  hasPart    Airblock_LSAGE_LSAGE_311LS_75_245_75_245
2  Airspace_LSAGE_411  hasPart  Airblock_LSAGE_LSAGE_503LS_155_245_155_245
3  Airspace_LSAGE_411  hasPart  Airblock_LSAGE_LSAGE_513LS_155_245_155_245
4  Airspace_LSAGE_411  hasPart  Airblock_LSAGE_LSAGE_716LF_155_245_155_245
5  Airspace_LSAGE_411  hasPart    Airblock_LSAGE_LSAGE_750LS_75_245_75_245
6  Airspace_LSAGE_411  hasPart    Airblock_LSAGE_LSAGE_778LS_75_245_75_245
7  Airspace_LSAGE_411  hasPart  Airblock_LSAGE_LSAGE_802LS_130_245_130_245
8  Airspace_LSAGE_411  hasPart     Airblock_LSAGE_LSAGE_804LS_75_245_75_24
9  Airspace_LSAGN_411  hasPart  Airblock_LSAGN_LSAGN_080LF_155_195_155_195
 
                    s        p                     o
0  Airspace_LSAGE_411  hasPart  Airblock_LSAGE_301LS
1  Airspace_LSAGE_411  hasPart  Airblock_LSAGE_311LS
2  Airspace_LS

RESULT TECHNICAL

As Giorgos suggested, the 109 store returns nicer Airblock strings. TODO recheck this query on 107 in the future to check if problems are fixed.

RESULT SEMANTICAL

The sub-Airspaces consist of airblocks. It should be possible to inspect these airblocks in more detail. Lets first check which properties are available for the Class FM_Airblock.

In [27]:
qry = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?p WHERE { ?s rdf:type :FM_Airblock ;   ?p ?o .} LIMIT 100 """

df = ts107.query(qry)
ts107.clean(df)
df.head(10)

,p
0,type
1,hasLowerLevel
2,hasUpperLevel
3,hasGeometry


RESULT SEMANTICAL

OK, so the airblocks should, as described in ontology and tutorial, be comprised out of :hasLowerLevel, :hasUpperLevel and :hasGeometry. Lets try to get these properties for the airblocks of the three Geneva airspaces LSAGE LSAGN and LSAGS.
The following query implements Giorgos recommendation in comment 14, to get the coordinates of all relevant airblocks. It is to be noted that Giorgos recommended to use the 109 endpoint. The query has slightly been altered because I am quering for a list of airblocks and not a list of airspaces.

PROBLEM

Why does this query not work with the 107 endpoint?

In [62]:
# Construct the list of airblocks that are to be passed 
str = ''
for i in range(len(blocks109)):
    if blocks109.iloc[i]['p'] == 'hasPart':
        str = str +' :' + blocks109.iloc[i]['o']
str = str[1:]
        
# check the list
print(str[:300] + '...')

:Airblock_LSAGE_301LS :Airblock_LSAGE_311LS :Airblock_LSAGE_503LS :Airblock_LSAGE_513LS :Airblock_LSAGE_716LF :Airblock_LSAGE_750LS :Airblock_LSAGE_778LS :Airblock_LSAGE_802LS :Airblock_LSAGE_804L :Airblock_LSAGN_080LF :Airblock_LSAGN_507LS :Airblock_LSAGN_700LF :Airblock_LSAGN_701LF :Airblock_LSAGN...


In [78]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT  ?airspace ?lowerlevel ?upperlevel ?block ?geom ?wkt
WHERE { 
   VALUES ?block { """ +str + """}
   ?airspace dul:hasPart       ?block .
   ?block       :hasLowerLevel ?lowerlevel ;
                :hasUpperLevel ?upperlevel ;
                :hasGeometry ?geom.
   ?geom        :hasMBR_WKT  ?wkt .
    
}
"""

df2 = ts109.query(qry)
df2 = ts109.clean(df2)
df2.head(10)

,airspace,lowerlevel,upperlevel,block,geom,wkt
0,Airspace_LSAGE_411,2286.0,7467.6,Airblock_LSAGE_301LS,http://83.212.239.107/geometries/airblocks/301LS,"POLYGON ((6.39666666666667 46.6491666666667, 6..."
1,Airspace_LSAGE_411,2286.0,7467.6,Airblock_LSAGE_311LS,http://83.212.239.107/geometries/airblocks/311LS,"POLYGON ((6.68444444444444 46.2133333333333, 6..."
2,Airspace_LSAGE_411,4724.400000000001,7467.6,Airblock_LSAGE_503LS,http://83.212.239.107/geometries/airblocks/503LS,"POLYGON ((6.32888888888889 46.4380555555556, 6..."
3,Airspace_LSAGE_411,4724.400000000001,7467.6,Airblock_LSAGE_513LS,http://83.212.239.107/geometries/airblocks/513LS,"POLYGON ((6.40305555555556 46.3416666666667, 6..."
4,Airspace_LSAGE_411,4724.400000000001,7467.6,Airblock_LSAGE_716LF,http://83.212.239.107/geometries/airblocks/716LF,"POLYGON ((6.39166666666667 46.7433333333333, 6..."
5,Airspace_LSAGE_411,2286.0,7467.6,Airblock_LSAGE_750LS,http://83.212.239.107/geometries/airblocks/750LS,"POLYGON ((6.64166666666667 46.9808333333333, 6..."
6,Airspace_LSAGE_411,2286.0,7467.6,Airblock_LSAGE_778LS,http://83.212.239.107/geometries/airblocks/778LS,"POLYGON ((7.12305555555556 46.3925, 7.12305555..."
7,Airspace_LSAGE_411,3962.4,7467.6,Airblock_LSAGE_802LS,http://83.212.239.107/geometries/airblocks/802LS,"POLYGON ((7.20138888888889 46.1833333333333, 7..."
8,Airspace_LSAGN_411,4724.400000000001,5943.6,Airblock_LSAGN_080LF,http://83.212.239.107/geometries/airblocks/080LF,"POLYGON ((5.6 46.1166666666667, 5.6 46.5461111..."
9,Airspace_LSAGN_411,5943.6,7467.6,Airblock_LSAGN_507LS,http://83.212.239.107/geometries/airblocks/507LS,"POLYGON ((5.97222222222222 46.1433333333333, 5..."





The query below gives us all spanish airblocks with their coordinates, the airspace thy belong to and the super-airspace they might belong to.

RESULT SEMANTICAL

An airblock may belong to several super-airspaces, depending in the configuration. For example, we can often see that an airblock belongs to Airspace_LETOT_411, which I guess means "Spain Total". Therefore, this query returns 4195 results, but there are only < 1000 unique airblocks in spain. But we can let Python-Pandas do the filtering later on, if needed.

In [83]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX myfn: <java:datAcronTester.unipi.gr.sparql_functions.>

SELECT  ?bigairspace ?airspace ?block ?wkt ?lowerlevel ?upperlevel 
WHERE { 
   
   ?airspace dul:hasPart       ?block .
   ?block       :hasLowerLevel ?lowerlevel ;
                :hasUpperLevel ?upperlevel ;
                :hasGeometry   ?geom.
   ?geom        :hasMBR_WKT  ?wkt .
   
   OPTIONAL {?bigairspace dul:hasPart ?airspace.}.
   
   FILTER regex(str(?block), 'Airblock_LE', "i")
}
"""

df2 = ts109.query(qry)
df2 = ts109.clean(df2)
%time df2 = df2.sort_values('block')

df2.to_csv('data/allspanishairblocks.csv')
df2.head(10)


CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 6.34 ms


,bigairspace,airspace,block,wkt,lowerlevel,upperlevel
0,Airspace_LETOT_411,Airspace_LEABTA_411,Airblock_LEABTA_249LE,"POLYGON ((-2.99166666666667 39, -2.99166666666...",0.0,7467.6
1,Airspace_LEABTMA_411,Airspace_LEABTA_411,Airblock_LEABTA_249LE,"POLYGON ((-2.99166666666667 39, -2.99166666666...",0.0,7467.6
2,Airspace_LETOT_411,Airspace_LEAMTA_411,Airblock_LEAMTA_650LE,"POLYGON ((-2.84083333333333 36.4166666666667, ...",0.0,4419.6
3,Airspace_LEAMTMA_411,Airspace_LEAMTA_411,Airblock_LEAMTA_650LE,"POLYGON ((-2.84083333333333 36.4166666666667, ...",0.0,4419.6
4,Airspace_LETOT_411,Airspace_LEAMTA_411,Airblock_LEAMTA_651LE,"POLYGON ((-1.83333333333333 36.85, -1.83333333...",0.0,4419.6
5,Airspace_LEAMTMA_411,Airspace_LEAMTA_411,Airblock_LEAMTA_651LE,"POLYGON ((-1.83333333333333 36.85, -1.83333333...",0.0,4419.6
6,Airspace_LETOT_411,Airspace_LEAMTA_411,Airblock_LEAMTA_652LE,"POLYGON ((-1.86111111111111 36.7138888888889, ...",0.0,4419.6
7,Airspace_LEAMTMA_411,Airspace_LEAMTA_411,Airblock_LEAMTA_652LE,"POLYGON ((-1.86111111111111 36.7138888888889, ...",0.0,4419.6
9,Airspace_LEAMTMA_411,Airspace_LEAMTA_411,Airblock_LEAMTA_653LE,"POLYGON ((-1.92361111111111 36.4530555555556, ...",0.0,4419.6
8,Airspace_LETOT_411,Airspace_LEAMTA_411,Airblock_LEAMTA_653LE,"POLYGON ((-1.92361111111111 36.4530555555556, ...",0.0,4419.6


Now we have the pure geometry of the airblocks. What I want to achieve now is to align this geometry with the regulation informations.


### Get frequency of configurations applied (Page 3)

This query returns the number of configurations that have been applied to the sector which are available in the database.


In [77]:
qry = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT (COUNT(?x) as ?count) ?a
WHERE {
    ?x :configurationOfAirspace ?a ;
       :hasCapacity ?capacity .
    FILTER(?capacity != "999")
} 
GROUP BY ?a
ORDER BY DESC(?count)
"""
df = ts107.query(qry)
ts107.clean(df)
#df.to_csv("airspaceconfigs.csv")
df.head(10)

,count,a
0,60,Airspace_LEMDTMA_411
1,60,Airspace_LFMMCTAW_411
2,50,Airspace_LSAZUTA_411
3,40,Airspace_LECMCTAN_411
4,28,Airspace_LYBACTA_411
5,26,Airspace_LECMCTAS_411
6,26,Airspace_LOVVCTA_411
7,20,Airspace_LEBLTMA_411
8,20,Airspace_LECPCTA_411
9,18,Airspace_LECSCTA_411


### Get all configurations of an airspace in a specific time frame (Page 3)

The following query gets all configurationsof Airspace_LSAGCTA_411 on 27th of April, 2016 between 00:00 and  28th of April, 05:00 UTC. 



In [104]:
qry = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?config ?airspace ?capacity ?start ?end
WHERE {
    VALUES ?airspace {:Airspace_LECMCTAN_411}
    ?config a :FM_Configuration;
            :hasCapacity ?capacity ;
            :configurationOfAirspace ?airspace ;
            dul:hasConstituent ?t.
    ?t :TimeStart ?start ;
    :TimeEnd ?end .
    FILTER( str(?start)>"2016-04-27T00:00:00" && str(?end)<"2016-04-28T05:00:00" )
}
"""
df = ts107.query(qry)
ts107.clean(df)
df.to_csv("data/airspaceconfigs_0000_0500.csv")

df.head(50)

,config,airspace,capacity,start,end
0,AirspaceConfiguration_LECMCTAN_CNF3B_411,Airspace_LECMCTAN_411,31,2016-04-27T03:30:00,2016-04-27T04:44:00
1,AirspaceConfiguration_LECMCTAN_CNF3B_411,Airspace_LECMCTAN_411,31,2016-04-27T03:30:00,2016-04-27T04:44:00
2,AirspaceConfiguration_LECMCTAN_CNF3B_411,Airspace_LECMCTAN_411,31,2016-04-27T03:30:00,2016-04-27T04:44:00
3,AirspaceConfiguration_LECMCTAN_CNF3B_411,Airspace_LECMCTAN_411,31,2016-04-27T03:30:00,2016-04-27T04:44:00
4,AirspaceConfiguration_LECMCTAN_CNF5B_411,Airspace_LECMCTAN_411,31,2016-04-27T04:45:00,2016-04-27T05:29:00
5,AirspaceConfiguration_LECMCTAN_CNF5B_411,Airspace_LECMCTAN_411,31,2016-04-27T04:45:00,2016-04-27T05:29:00
6,AirspaceConfiguration_LECMCTAN_CNF5B_411,Airspace_LECMCTAN_411,31,2016-04-27T04:45:00,2016-04-27T05:29:00
7,AirspaceConfiguration_LECMCTAN_CNF5B_411,Airspace_LECMCTAN_411,31,2016-04-27T04:45:00,2016-04-27T05:29:00
8,AirspaceConfiguration_LECMCTAN_CNF8B2_411,Airspace_LECMCTAN_411,36,2016-04-27T13:00:00,2016-04-27T13:48:00
9,AirspaceConfiguration_LECMCTAN_CNF8B2_411,Airspace_LECMCTAN_411,36,2016-04-27T13:49:00,2016-04-27T20:29:00


In [102]:
qry = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?p
WHERE {
    ?s a :FM_Configuration ;
       ?p ?o  .
}
"""
df = ts109.query(qry)
ts109.clean(df)

df.head(15)

,p
0,type
1,hasCapacity
2,configurationOfAirspace
3,hasConstituent


## Multiple Trajectories Visualization (Page 3 bottom)

The following query pulls 100 trajectories out of the store, directly with their coordinates.

NOTE

Be aware that the direct availability of the coordinates is a convenience function created by the RDF maintainers. It prevents us from needing to traverse into every single semantic node of a tracjectory to get the coordinates and allows for easy plotting.

PROBLEM

Unfortunately, this convenience function only gives us 2D coordinates without altitude or time information. But what we need in order to cross trajectories and airblocks is the 4D coordinates including time and altitude. The chosen data format, well known text (WKT) would able to handle 4D coordinates, because z coordinate is available for altitude rep and m coordinate for linear data (time passed since epoch could be considered as a linear distance of a point from the epoch).


In [139]:
qry = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?flightPlanID ?trajectoryID ?dep ?dest ?wkt ?trajectoryType
WHERE {
  ?flightPlanID            :reportsTrajectory   ?trajectoryID;
                           :departureAirport    ?dep ;
                           :destinationAirport  ?dest .
  ?trajectoryID         dul:hasGeometry/:hasWKT ?wkt ;
                        a ?trajectoryType.
                        
  FILTER(?trajectoryType = :RegulatedTrajectory)
  
} LIMIT 100

"""

df = ts107.clean(ts107.query(qry))
df.to_csv("data/trajectories.csv")
df.head(5)

,flightPlanID,trajectoryID,dep,dest,wkt,trajectoryType
0,flight_plan_AA51123169,tr_20160401_670709_m2,Place_LyonSaintExupery_Airport,Place_ParisOrly_Airport,"LINESTRING(5.081111 45.725556, 5.077222 45.811...",RegulatedTrajectory
1,flight_plan_AA51131877,tr_20160401_687118_m2,Place_ParisOrly_Airport,Place_Porto___Pedras_Rubras,"LINESTRING(2.379444 48.723333, 2.365278 48.720...",RegulatedTrajectory
2,flight_plan_AA51132762,tr_20160401_687913_m2,Place_Zurich_Kloten,Place_FrankfurtMain_Airport,"LINESTRING(8.548056 47.458056, 8.523889 47.453...",RegulatedTrajectory
3,flight_plan_AA51134204,tr_20160401_689279_m2,Place_Hamburg_Airport,Place_LondonHeathrow_Airport,"LINESTRING(9.988333 53.630278, 9.981944 53.638...",RegulatedTrajectory
4,flight_plan_AA51136352,tr_20160401_691226_m2,Place_RaficHaririInternationalBeirut_Airport,Place_Istanbul_Ataturk_Airport,"LINESTRING(35.49 33.819167, 35.486111 33.8075,...",RegulatedTrajectory


#### Visualizing the trajectories 

We can visualize these trajectories in 2D using gemoet [https://pypi.python.org/pypi/geomet/0.1.0] for  WKT-->GeoJson transformation and folium [https://github.com/python-visualization/folium], which is a python wrapper for Leaflet.js interactive maps.

In [140]:
from geomet import wkt
import json

#Convert WKT column into a gejson column
df["geojson"] = df["wkt"].apply(lambda x: json.dumps(wkt.loads(x)))
df = df.drop('wkt', 1)

In [142]:
df.head(5)

,flightPlanID,trajectoryID,dep,dest,trajectoryType,geojson
0,flight_plan_AA51123169,tr_20160401_670709_m2,Place_LyonSaintExupery_Airport,Place_ParisOrly_Airport,RegulatedTrajectory,"{""type"": ""LineString"", ""coordinates"": [[5.0811..."
1,flight_plan_AA51131877,tr_20160401_687118_m2,Place_ParisOrly_Airport,Place_Porto___Pedras_Rubras,RegulatedTrajectory,"{""type"": ""LineString"", ""coordinates"": [[2.3794..."
2,flight_plan_AA51132762,tr_20160401_687913_m2,Place_Zurich_Kloten,Place_FrankfurtMain_Airport,RegulatedTrajectory,"{""type"": ""LineString"", ""coordinates"": [[8.5480..."
3,flight_plan_AA51134204,tr_20160401_689279_m2,Place_Hamburg_Airport,Place_LondonHeathrow_Airport,RegulatedTrajectory,"{""type"": ""LineString"", ""coordinates"": [[9.9883..."
4,flight_plan_AA51136352,tr_20160401_691226_m2,Place_RaficHaririInternationalBeirut_Airport,Place_Istanbul_Ataturk_Airport,RegulatedTrajectory,"{""type"": ""LineString"", ""coordinates"": [[35.49,..."


In [143]:
import folium


map1 = folium.Map(location=[40,10], zoom_start=4, control_scale=True, prefer_canvas=True)

for index, row in df.head(10).iterrows():
    c = folium.GeoJson(row['geojson'], name = (row['dep']+ row['dest']),overlay=True, 
                       style_function = lambda feature: {'fillColor': '#ffaf00','color': 'blue', 'weight': 2.5,'dashArray': '5, 5'},
                       highlight_function = lambda feature: {'fillColor': '#ffaf00','color': 'green', 'weight': 5,'dashArray': '5, 5'})
    c.add_child(folium.Popup(row['trajectoryID'] + '\n' + row['dep'] +'\n' + row['dest']))
    c.add_to(map1)
folium.LayerControl().add_to(map1)
map1.save(outfile='maps/map1.html')

map1    

## Single Trajectory Inspection (Page 4 bottom + Comment 24 by Giorgos)

According to the tutorial, the following query should give us all semantic nodes and their linked information, for one specific trajectory.

This section has been updated with the comment 24 by Giorgos. The query has been updated, and, the naming convention of the trajectories has changed. The trajectories that resulted in 10,000+ nodes before, like `:traj_AA51114336_20160331154500` between Bristol and Tenerife, do not exist anymore in the dataset. 
The first query below returns proper results with 66 nodes for a single trajectory.

The final goal is to recosntruct the rajectories from the nodes, in order to have "enriched" trajectories with time, altitude and weather information. For this, we try to get all positions and times into one GeoJSON file which can then be passed to the folium map. According to their tutorial, the format of the GeoJSON is as follows:

```
{
"type": "Feature",
"geometry": {"type": "LineString", 
             "coordinates": [[lon, 25] for
                                lon in np.linspace(-150, 150, 25)] }, 
"properties": {"times": [1435708800000+i*86400000 for i in np.linspace(0, 25, 25)] }
}
```
So we need to create a dict which is formatted accordingly. The functions defined in the following box will help me with this task.

In [197]:
from geomet import wkt
import json
from datetime import datetime

def wkt_to_geojson(dataframe, wkt_column_name, geojson_column_name):
    """
    Converts a WKT column into a GeoJSON column.
    The GeoJSON column then contains a dict.
    
    Keyword arguments:
    @param dataframe: the pandas.DataFrame which should be altered
    @param wkt_column_name: the name of the column containing the well-known-text
    @param geojson_column_name: the desired geoJSON column name
    
    """
    if type(dataframe) != pd.core.frame.DataFrame:
        raise TypeError('The parameter dataframe must be a Pandas DataFrame.')
    if not wkt_column_name in dataframe.columns:
        raise ValueError('No column with the specified name found in the DataFrame')
    
    dataframe[geojson_column_name] = dataframe[wkt_column_name].apply(lambda x: wkt.loads(x))
    dataframe = dataframe.drop(wkt_column_name, 1)
    return dataframe


def geojsonify(dataframe, time_column, coord_column):
    """
    Given a set of nodes as a DataFrame, this function tterates through the rows 
    and creates a python dict in geoJSON format.
    """   
    res = {'type': 'Feature',
           'geometry': {'type': 'LineString',
                        'coordinates': [],
                        'properties': {'times': [] }
                       }
          }
    for x in range(len(dataframe)):
        res['geometry']['coordinates'].append(dataframe.iloc[x][coord_column])
        res['geometry']['properties']['times'].append(dataframe.iloc[x][time_column])
    return res 


def inject_time(geojsonpoint, time):
    """
    Injects Time dimension into geoJSON coordinates. 
    @param geojsonpoint: a dict in geoJSON point format
    @param time: a timestring
    """
    #geojson['coordinates'] = [geojson['coordinates'][0], geojson['coordinates'][1], time]
    geojson['properties'] = {'times' : [time]}
    return geojson


def extract_coordinates(geojson):
    """
    Returns only the coordinates from a geoJSON POINT object
    """
    return geojson['coordinates']

def timestring_to_epoch(timestring):
    """
    Converts a datAcron formatted timestring into milliseconds since epoch.
    """
    return datetime.strptime(timestring, '%Y-%m-%dT%H:%M:%S').timestamp()

Lets now see the updated and proper query of a single trajectory.

In [206]:
qry = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?node ?geom ?time  
WHERE {
  :tr_20160402_711655_m2 dul:hasPart ?node .
  ?node                     :hasGeometry/:hasWKT ?geom ;
                         dul:hasTemporalFeature/:TimeStart ?time.
  OPTIONAL{?node :hasWeatherCondition ?w .}.
} ORDER BY ?time

"""

singletrajectory = ts107.clean(ts107.query(qry))

print(len(singletrajectory))
singletrajectory.head(5)

66


,node,geom,time
0,n_m2_20160402_711655_1,POINT(2.738889 39.551667),2016-04-02T08:25:00
1,n_m2_20160402_711655_2,POINT(2.805 39.582778),2016-04-02T08:26:27
2,n_m2_20160402_711655_3,POINT(2.8425 39.600556),2016-04-02T08:26:57
3,n_m2_20160402_711655_4,POINT(2.899167 39.627222),2016-04-02T08:27:43
4,n_m2_20160402_711655_5,POINT(2.951111 39.648333),2016-04-02T08:28:19


In [222]:
qry = """ 
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?o
WHERE {
  :tr_20160402_711655_m2 a ?o
} 

"""

singletrajectory = ts107.clean(ts107.query(qry))

print(len(singletrajectory))
singletrajectory.head()

1


,o
0,RegulatedTrajectory


In [199]:
#convert position to geoJson
singletrajectory = wkt_to_geojson(singletrajectory, 'geom', 'geojson')

#pull only coordinates out of the GeoJSON column
singletrajectory['coord_only'] =  singletrajectory['geojson'].apply(lambda x: extract_coordinates(x))
singletrajectory.head(5)

#convert time string into milliseconds since epoch. After this, we can sort the resulting DataFrame by time.
singletrajectory['time'] = singletrajectory['time'].apply(lambda x: timestring_to_epoch(x))
singletrajectory = singletrajectory.sort_values('time')


singletrajectory.head(10)

,node,time,geojson,coord_only
0,n_m2_20160402_711655_1,1.459578e+09,"{'type': 'Point', 'coordinates': [2.738889, 39...","[2.738889, 39.551667]"
1,n_m2_20160402_711655_2,1.459578e+09,"{'type': 'Point', 'coordinates': [2.805, 39.58...","[2.805, 39.582778]"
2,n_m2_20160402_711655_3,1.459578e+09,"{'type': 'Point', 'coordinates': [2.8425, 39.6...","[2.8425, 39.600556]"
3,n_m2_20160402_711655_4,1.459578e+09,"{'type': 'Point', 'coordinates': [2.899167, 39...","[2.899167, 39.627222]"
4,n_m2_20160402_711655_5,1.459578e+09,"{'type': 'Point', 'coordinates': [2.951111, 39...","[2.951111, 39.648333]"
5,n_m2_20160402_711655_6,1.459579e+09,"{'type': 'Point', 'coordinates': [3.261944, 39...","[3.261944, 39.775]"
6,n_m2_20160402_711655_7,1.459579e+09,"{'type': 'Point', 'coordinates': [3.324722, 39...","[3.324722, 39.816389]"
7,n_m2_20160402_711655_8,1.459579e+09,"{'type': 'Point', 'coordinates': [3.396667, 39...","[3.396667, 39.863333]"
8,n_m2_20160402_711655_9,1.459579e+09,"{'type': 'Point', 'coordinates': [3.558333, 39...","[3.558333, 39.969444]"
9,n_m2_20160402_711655_10,1.459579e+09,"{'type': 'Point', 'coordinates': [3.657222, 40...","[3.657222, 40.034444]"


Now we have the times and the coordinates in a dict-friendly way. We can now iterate through the complete dataset (rows) and create our custom GeoJSON object, that we can then pass to the folium map. 

In [203]:
from folium.plugins import TimestampedGeoJson

somegeojson = geojsonify(singletrajectory, 'time', 'coord_only')

map2 = folium.Map(location=[40,10], zoom_start=4, control_scale=True, prefer_canvas=True)

#tgj = TimestampedGeoJson(data = somegeojson, period = 'PT1M')
#map2.add_child(tgj, name='sometimestampedgeojson')

c = folium.GeoJson(somegeojson, name = 'Traj with time',overlay=True, 
                       style_function = lambda feature: {'fillColor': '#ffaf00','color': 'blue', 'weight': 2.5,'dashArray': '5, 5'},
                       highlight_function = lambda feature: {'fillColor': '#ffaf00','color': 'green', 'weight': 5,'dashArray': '5, 5'})
c.add_child(folium.Popup('some popup'))
c.add_to(map2)

folium.LayerControl().add_to(map2)
map2.save(outfile='maps/map2.html')

map2  

In [219]:
import cesiumpy

v = cesiumpy.Viewer()
#b = cesiumpy.Box(dimensions=(40e4, 30e4, 50e4), material=cesiumpy.color.RED, position=[-120, 40, 0])
#v.entities.add(b)

ds = cesiumpy.CzmlDataSource('data/simple.czml')
v.dataSources.add(ds)

v




/home/joerg/anaconda3/envs/datacron1/lib/python3.6/site-packages/cesiumpy/util/html.py:14: UserWarning: Unable to read specified path, be sure to the output HTML can read the path: 
  warnings.warn(msg.format(sourceUri))


# Experiments


In [243]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?p
WHERE {
  ?s a :ATC_WeatherRegulation ;
     ?p ?o .
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = ts107.query(qry)
df = ts107.clean(df)
print(len(df))
df.head(10)

6


,p
0,type
1,RegulationAiracCycle
2,RegulationDescription
3,hasRegion
4,hasParticipant
5,hasTimeInterval


In [246]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?s ?airac ?desc ?region ?time
WHERE {
  ?s a  :ATC_WeatherRegulation ;
        :RegulationAiracCycle ?airac ;
        :RegulationDescription ?desc ;
        :hasRegion ?region ;
     dul:hasParticipant ?participant ;
     dul:hasTimeInterval ?time .
     
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = ts107.query(qry)
df = ts107.clean(df)
print(len(df))
df.head(10)

177


,s,airac,desc,region,time
0,LFPOA01E_411,411,LOW CEILING,Sector_LFFFAD,intv_1459489200000_1459489200000
1,LFPOA01M_411,411,LOW CEILING,Sector_LFFFAD,intv_1459503600000_1459503600000
2,LSZHA01M_411,411,WIND,Sector_LSAZFMP,intv_1459498800000_1459498800000
3,LFPOA03_411,411,TS,Sector_LFFFAD,intv_1459711200000_1459711200000
4,EGGWA04M_411,411,LOW VISIBILITY,Sector_EGTCFMP,intv_1459749600000_1459749600000
5,EGSSA04M_411,411,LOW VISIBILITY,Sector_EGTCFMP,intv_1459749600000_1459749600000
6,GCXOA05M_411,411,LOW CEILING,Sector_GCCCFMP,intv_1459842000000_1459842000000
7,EGSSA01_411,411,FORECASTED LVP,Sector_EGTCFMP,intv_1459490400000_1459490400000
8,LEBLA01M_411,411,CB + STORM,Sector_LECBFMP,intv_1459490400000_1459490400000
9,LSZHA01_411,411,WIND,Sector_LSAZFMP,intv_1459516800000_1459516800000


In [271]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?reginstance ?tr ?region ?p ?o
WHERE {
  #?regtype rdfs:subClassOf* :FM_Regulation .
  #?reginstance a ?regtype .
  #?tr :affectedBy ?reginstance .
  ?reginstance rdf:type/rdfs:subClassOf* :FM_Regulation .
  ?tr :affectedBy ?reginstance .
  ?reginstance :hasRegion ?region .
  ?region ?p ?o .

} 
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = ts107.query(qry)
df = ts107.clean(df)
print(len(df))
df.head(10)

0


,reginstance,tr,region,p,o


In [273]:
qry = """
PREFIX : <http://www.datacron-project.eu/datAcron#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT DISTINCT ?p ?o
WHERE {
 :Airblock_917ED ?p ?o.
}
"""
 #?s rdf:type/rdfs:subClassOf* :SpatiotemporalRegion 

df = ts107.query(qry)
df = ts107.clean(df)
print(len(df))
df.head(10)

0


,p,o
